# Plot example data for a single patient

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
%matplotlib inline

In [ ]:
# Config
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

In [ ]:
# Connect to MIMIC
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# cur.close()
# con.close()

In [ ]:
# Select a single ICU stay
icustay_id = 217082

In [ ]:
# Load admissions
query = """
SELECT ad.subject_id
  , ad.hadm_id
  , ad.admission_type
  , ad.diagnosis
  , ic.icustay_id
  , ic.first_careunit
  , ic.last_careunit
  , ic.intime as icu_intime
  , ad.hospital_expire_flag
  , pa.expire_flag
FROM admissions ad
INNER JOIN icustays ic
ON ad.subject_id = ic.subject_id
INNER JOIN patients pa
ON ad.subject_id = pa.subject_id
WHERE ic.icustay_id = """ + str(icustay_id) + """ 
ORDER BY ic.intime
"""

admissions = pd.read_sql_query(query,con)

In [ ]:
# First few rows of admissions
admissions.head()

In [ ]:
# Load chartevents
query = """
SELECT ce.icustay_id
  , ce.charttime
  , ce.charttime - ie.intime AS icutime
  , di.label
  , ce.value
  , ce.valuenum
  , ce.valueuom
FROM chartevents ce
INNER join d_items di
ON ce.itemid = di.itemid
INNER join icustays ie
ON ce.icustay_id = ie.icustay_id
WHERE ce.icustay_id = """ + str(icustay_id) + """
AND error != 1
ORDER BY ce.charttime
"""

charts = pd.read_sql_query(query,con)

In [ ]:
# First few rows of charts
charts.head()

In [ ]:
# Load outputevents
query = """
SELECT oe.icustay_id
  , oe.charttime
  , oe.charttime - ie.intime AS icutime
  , di.label
  , oe.value
  , oe.valueuom
FROM outputevents oe 
INNER JOIN icustays ie
  ON oe.icustay_id = ie.icustay_id
INNER JOIN d_items di
  ON oe.itemid = di.itemid
WHERE oe.icustay_id = """ + str(icustay_id) + """
ORDER BY oe.charttime
"""

outputs = pd.read_sql_query(query,con)

In [ ]:
# First few rows of outputs
outputs.head()

In [ ]:
# Load inputevents
query = """
SELECT inp.icustay_id
  , inp.starttime
  , inp.endtime
  , inp.starttime - ie.intime AS icustarttime
  , inp.endtime - ie.intime AS icuendtime
  , di.label
  , inp.amount
  , inp.amountuom
  , inp.rate
  , inp.rateuom
FROM inputevents_mv inp 
INNER join icustays ie
  ON inp.icustay_id = ie.icustay_id
INNER join d_items di
  ON inp.itemid = di.itemid
WHERE inp.icustay_id = """ + str(icustay_id) + """
AND lower(inp.statusdescription) != 'rewritten'
ORDER BY inp.starttime
"""

inputs = pd.read_sql_query(query,con)

In [ ]:
# First few rows of inputs
inputs.head()

In [ ]:
# Load labevents
query = """
SELECT le.subject_id
  , ie.icustay_id
  , le.charttime
  , le.charttime - ie.intime AS icutime
  , di.label
  , le.value
  , le.valuenum
  , le.valueuom
FROM labevents le
INNER join icustays ie
  ON le.subject_id = ie.subject_id
  AND le.charttime >= ie.intime
  AND le.charttime <= ie.outtime
INNER JOIN d_labitems di
  ON le.itemid = di.itemid
WHERE ie.icustay_id = """ + str(icustay_id) + """
ORDER BY le.charttime
"""

labs = pd.read_sql_query(query,con)

In [ ]:
# First few rows of labs
labs.head()

In [ ]:
# Snip data to first 24 hours
charts = charts.loc[charts.icutime.dt.days<3]
outputs = outputs.loc[outputs.icutime.dt.days<3]
inputs = inputs.loc[inputs.icustarttime.dt.days<3]
labs = labs.loc[labs.icutime.dt.days<3]

In [ ]:
# Create column with minutes from ICU intime
charts['icutimehr'] = (charts['icutime'].dt.seconds/60/60)+(charts['icutime'].dt.days*24)
outputs['icutimehr'] = (outputs['icutime'].dt.seconds/60/60)+(charts['icutime'].dt.days*24)
inputs['icustarttimehr'] = (inputs['icustarttime'].dt.seconds/60/60)+(charts['icustarttimehr'].dt.days*24)
inputs['icuendtimehr'] = (inputs['icuendtime'].dt.seconds/60/60)+(charts['icuendtime'].dt.days*24)
labs['icutimehr'] = (labs['icutime'].dt.seconds/60/60)+(charts['icutime'].dt.days*24)

In [ ]:
# What are the most commonly measured variables in charts?
charts['label'].value_counts().head()

In [ ]:
# What are the most commonly measured variables in charts?
outputs['label'].value_counts().head()

In [ ]:
# What are the most commonly measured variables in charts?
inputs['label'].value_counts().head()

In [ ]:
# What are the most commonly measured variables in charts?
labs['label'].value_counts().head()

In [ ]:
# Plot sample data over first 24 hours from admission to ICU
# Credit: Randal Olson for styling (http://www.randalolson.com/2014/06/28/)

# Prepare the size of the figure
fig = plt.figure(figsize=(19, 15))
plt.rcParams.update({'font.size': 22})

# "Tableau 20" colors as RGB.   
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]  
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)
    
# Remove the plot frame lines. 
ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
ax.spines["bottom"].set_visible(True)    
ax.spines["right"].set_visible(False)    
ax.spines["left"].set_visible(True)    
  
# Ensure that the axis ticks only show up on the bottom and left of the plot.      
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

# Plot a few vital signs
plt.plot(charts.icutimehr[charts.label=='Heart Rate'], 
         charts.valuenum[charts.label=='Heart Rate'],
         color=tableau20[1], lw=2.5,
         marker='o', markersize=6, label='Heart rate')

plt.plot(charts.icutimehr[charts.label=='Respiratory Rate'], 
         charts.valuenum[charts.label=='Respiratory Rate'],
         color=tableau20[2], lw=2.5,
         marker='o', markersize=6, label='Resp. rate')

plt.plot(charts.icutimehr[charts.label=='Non Invasive Blood Pressure mean'], 
         charts.valuenum[charts.label=='Non Invasive Blood Pressure mean'],
         color=tableau20[3], lw=2.5,
         marker='o', markersize=6, label='BP mean')

# plt.plot(charts.icutimehr[charts.label=='Non Invasive Blood Pressure systolic'], 
#          charts.valuenum[charts.label=='Non Invasive Blood Pressure systolic'],
#          color=tableau20[3], lw=2.5, 
#          marker='o', markersize=6, label='BP sys.')

# plt.plot(charts.icutimehr[charts.label=='Non Invasive Blood Pressure diastolic'], 
#          charts.valuenum[charts.label=='Non Invasive Blood Pressure diastolic'],
#          color=tableau20[4], lw=2.5, 
#          marker='o', markersize=6, label='BP dias.')

plt.plot(charts.icutimehr[charts.label=='Temperature Fahrenheit'], 
         (charts.valuenum[charts.label=='Temperature Fahrenheit']-32)/1.8,
         color=tableau20[5], lw=2.5, 
         marker='o', markersize=6, label='Temp.')

plt.plot(charts.icutimehr[charts.label=='O2 saturation pulseoxymetry'], 
         charts.valuenum[charts.label=='O2 saturation pulseoxymetry'],
         color=tableau20[6], lw=2.5, 
         marker='o', markersize=6, label='O2 sat.')

# Plot some laboratory values
plt.plot(labs.icutimehr[labs.label=='HEMATOCRIT'], 
         labs.valuenum[labs.label=='HEMATOCRIT'], 
         color=tableau20[7], lw=2.5, 
         marker='o', markersize=6, label='Haematocrit')

plt.plot(labs.icutimehr[labs.label=='HEMOGLOBIN'], 
         labs.valuenum[labs.label=='HEMOGLOBIN'], 
         color=tableau20[8], lw=2.5,
         marker='o', markersize=6, label='Hemoglobin')

# Plot some input/output events
plt.plot(inputs.icustarttimehr[inputs.amountuom=='mL'], 
         inputs.amount[inputs.amountuom=='mL'].cumsum()/100, 
         color=tableau20[9], lw=2.5,
         marker='o', markersize=6, label='Intake volume, dL')

plt.plot(outputs.icutimehr, 
         outputs.value.cumsum()/100, 
         color=tableau20[10], lw=2.5,
         marker='o', markersize=6, label='Output volume, dL')

# Plot Glasgow Coma Scale measurements every hour or so
# plt.text(-10,155,'GCS - Eye Opening',fontsize=14)
for i, txt in enumerate(charts.value[charts.label=='GCS - Eye Opening'].values):
    if np.mod(i,2)==0 and i < 65:
        plt.annotate(txt, (charts.icutimehr[charts.label=='GCS - Eye Opening'].values[i],155),fontsize=15)

# plt.text(-60,150,'GCS - Motor Response',fontsize=14)
for i, txt in enumerate(charts.value[charts.label=='GCS - Motor Response'].values):
    if np.mod(i,2)==0 and i < 65:
        plt.annotate(txt, (charts.icutimehr[charts.label=='GCS - Motor Response'].values[i],150),fontsize=15)

# plt.text(-60,145,'GCS - Verbal Response',fontsize=14)  
for i, txt in enumerate(charts.value[charts.label=='GCS - Verbal Response'].values):
    if np.mod(i,2)==0 and i < 65:
        plt.annotate(txt, (charts.icutimehr[charts.label=='GCS - Verbal Response'].values[i],145),fontsize=15)

plt.legend(loc=5,fontsize=18)
plt.xlabel('Time from admission to ICU, hours', fontsize=22)
plt.ylabel('Measurement, absolute value', fontsize=22)
plt.ylim(0,160) 

# Save the figure
# fig.savefig('examplepatient.pdf', bbox_inches='tight')